In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from functools import partial


# Python y rendimiento | Parte 2

Python es un lenguaje versátil pero poco eficiente comparado a lenguajes de bajo nivel (C/Fortran)

Esto puede ser perjudicial si queremos usar Python para procesar una base de datos extensa o entrenar un modelo con una gran cantidad de parámetros

Sin embargo, lo que es sumamente costoso para un código escrito en "Python puro" no lo es tanto cuando usamos las librerías de cómputo científico de Python que hemos visto en este curso 

Las funciones de **Numpy**, **Scipy** y **Pandas** son eficientes porque están basadas en rutinas escritas en lenguaje C y Fortran

> La clave para lograr un alto rendimiento es usar adecuadamente las librerías

Cuando usamos estas librerías estamos usando código compilado en lugar de interpretado 

> Existen maneras de conectar nuestro código Python con código compilado escrito en lenguaje C o Fortran

En este clase veremos tres formas para mejorar el rendimiento de una rutina matemática arbitraria escrita en "Python puro":

1. Cómputo basado en arreglos con `NumPy`
1. Uniendo Python y C con `Cython`
1. Aprovechando arquitecturas multi-nucleo con `Multiprocessing`

Esto se enmarca en el contexto de **optimización de software**


## ¿Qué es la optimización de códigos/software?

Se refiere a modificar una rutina computacional para mejorar su eficiencia, es decir reducir sus

1. Tiempos de ejecución
1. Consumo de recursos 

El aspecto que se intenta modificar es aquel que limita nuestro programa 

Podemos hablar entonces de programas que están limitados en cómputo (compute-bound), limitados en memoría (memory-bound) 

La optimización puede ocurrir en distintos niveles

En el ámbito de la computación científica lo más común es enfrentar programas que están límitados... 



### ¿Por qué optimizar?



### ¿Cuándo optimizar?

`Si:` 

    tu rutina está incompleta o no entrega el resultado esperado
    
`Entonces:`

    No es momento de optimizar 


Considera que optimizar puede 
- hacer el código más complicado y menos legible 
- introducir bugs
- tomar tiempo y bastante dedicación

Por lo tanto debemos evitar optimizar de forma prematura

> Premature optimization is the root of all evil - Donald Knuth 


### ¿Dónde optimizar?

Evita gastar tiempo optimizando rutinas que influyan poco en el rendimiento total del programa

La optimización debería concentrarse en las secciones más lentas y/o costosas

Podemos encontrar dichas secciones haciendo un *profiling* de nuestro código


### 2. Haz buen uso de la sintaxis y funciones de Python

https://ipython-books.github.io/51-knowing-python-to-write-faster-code/

https://wiki.python.org/moin/PythonSpeed/PerformanceTips

http://people.duke.edu/~ccc14/sta-663-2017/10A_CodeOptimization.html


# Cómputo basado en arreglos con Numpy


Getting good performance out of code utilizing NumPy is often straightforward, asarray operations typically replace otherwise comparatively extremely slow pure Pythonloops. Here is a brief list of some of the things to keep in mind:• Convert Python loops and conditional logic to array operations and boolean arrayoperations• Use broadcasting whenever possible• Avoid copying data using array views (slicing)• Utilize ufuncs and ufunc methods



### Ejemplo: Vectorizando el cálculo del "Set de Julia"

Consideremos el código en "Python puro" que vimos en la clase de *profiling*

In [ ]:
def evaluate(zi, zr, maxiters=50, cr=-0.835, ci=-0.2321):
    nit = 0
    zi2 = zi**2
    zr2 = zr**2
    while zi2 + zr2 <= 4. and nit < maxiters:
        zi = 2*zr*zi + ci
        zr = zr2 - zi2 + cr
        zr2 = zr**2
        zi2 = zi**2 
        nit +=1
    return nit
    
def make_fractal(N, maxiters=50):
    image = []
    for i in range(N):
        row = []
        for j in range(2*N):
            zi = -1.0 + i*2/N
            zr = -2.0 + j*2/N
            row.append(evaluate(zi, zr, maxiters))
        image.append(row)
    return image

plt.figure(figsize=(8, 4), tight_layout=True)
plt.gca().axis('off')
plt.imshow(make_fractal(500));

El tiempo total para calcular el fractal con una resolución de 500x1000 es

In [ ]:
%timeit -r3 -n1 make_fractal(500)

Estudiando el código nos damos cuenta que la función `evaluate` se ejecuta de forma secuencial para cada uno de los 500x1000 píxeles

Si guardamos la imagen y los valores de $z$ como un `ndarray` podemos actualizar todos los pixeles "al mismo tiempo"

In [ ]:
def make_fractal_vectorized(N, maxiters=50):
    image = np.zeros(shape=(N, 2*N), dtype=int)
    zi = np.linspace(-1, 1, num=N, endpoint=False)[:, None]
    zr = np.linspace(-2, 2, num=2*N, endpoint=False)[:, None]
    zi = np.repeat(zi, repeats=2*N, axis=1)
    zr = np.repeat(zr.T, repeats=N, axis=0)
    cr, ci = -0.835, -0.2321
    nit = 0
    while nit < maxiters:
        zr2 = zr**2
        zi2 = zi**2
        mask = zr2 + zi2 <= 4.
        image[mask] += 1
        zi[mask] = 2*zr[mask]*zi[mask] + ci
        zr[mask] = zr2[mask] - zi2[mask] +  cr
        nit += 1
    return image

np.allclose(make_fractal(500), make_fractal_vectorized(500))

El resultado es idéntico al código secuencial en "Python puro"

El tiempo total para calcular el fractal usando el código vectorizado es:

In [ ]:
%timeit -r3 -n1 make_fractal_vectorized(500)

El *speed-up* es el tiempo de la nueva rutina dividido el tiempo de referencia (rutina secuencial)

¿Cuánto es el *speed-up* en este caso?

#### Propuesto

En cada iteración del `while` calculamos el cuadrado de todos los $z$ 

Sin embargo lo más correcto sería calcular los cuadrados solo para los $z$ que cumplieron con la condición $|z| < 4$ la iteración anterior

Considera esta observación para optimizar aun más el código de `make_fractal_vectorized` ¿Cuánto *speed-up* obtienes?

# CPython 

La [implementación](https://wiki.python.org/moin/PythonImplementations) de Python más utilizada está escrita en C y se llama CPython

La implementación es aquello que interpreta y corre el código escrito en Python

También existen implementaciones en Java ([Jython](https://www.jython.org/)) y C# (IronPython)

CPython compila el código escrito en Python en un código de máquina (binario). Luego CPython interpreta el binario

Podemos estudiar dicho código usando el módulo [dis](https://docs.python.org/3.7/library/dis.html) de Python

In [ ]:
import dis
dis.dis(make_fractal)

# [Cython](https://cython.org/): C Extensions for Python

Python es un lenguaje interpretado con tipos dinámicos. Esto hace que cada operación tenga un overhead.

Cython es un lenguaje de programación que combina Python con el sistema de tipos estáticos de C y C++ eliminando los típicos overheads de Python

El compilador de Cython convierte el código fuente en código C que a su vez se compila como un módulo de Python con CPython

> Una vez compilado el código escrito en Cython puede llamarse desde Python

¿Por qué considerar Cython?
- Cython es casi tan simple como Python
- Cython es casi tan rápido como C
- Cython se integra de buena manera con NumPy
- Con Cython se pueden llamar librerías de C

### Ejemplo: Distancia euclidiana todos-con-todos

Sea un conjunto de 1000 datos bidimensionales

Queremos calcular la distancia euclidiana de cada dato con todos los demás, es decir una matriz donde el elemento $ij$ es 

$$
d_{ij} = \sum_{k=1}^2 (x_{ik} - x_{jk})^2
$$

A continuación se muestra dos códigos que cumplen este proposito y obtienen un resultado equivalente

- El primero usa "Python puro" y calcula las distancias de forma secuencial
- El segundo usa operaciones vectoriales de NumPy y calcula las distancias "al mismo tiempo"

Estudie ambos códigos hasta comprenderlos

In [ ]:
data = np.random.randn(1000, 2)

def distancia_pares(data):
    N = len(data)
    D = np.empty(shape=(N, N))
    for i in range(N):
        for j in range(N):
            D[i, j] = sum((data[i] - data[j])**2)
    return D
            
def distancia_pares_numpy(data):
    return np.sum((data.reshape(-1, 1, 2) - data.reshape(1, -1, 2))**2, axis=-1)

display(np.allclose(distancia_pares(data), distancia_pares_numpy(data)))
%timeit -r3 -n1 distancia_pares(data)
%timeit -r3 -n1 distancia_pares_numpy(data)

Ahora escribamos una versión en Cython de la función `distancia_pares`

En Jupyter podemos cargar la extensión `cython` y luego usar la magia de bloque `%%cython` 

- Esto hace que el bloque acepte lenguaje cython y que al ejecutarlo se compile
- Luego podremos llamar la función `distancia_pares_cython` desde bloques regulares de Python
- Los errores de compilación aparecen como la salida del bloque
- La opción `-a` (annotate) retorna un profile linea a linea indicando con amarillo las llamadas a CPython (mientras más llamadas más lento es nuestro código)

In [ ]:
%load_ext cython

In [ ]:
%%cython -a
import numpy as np
def distancia_pares_cython(data):
    N = len(data)
    D = np.empty(shape=(N, N))
    for i in range(N):
        for j in range(N):
            D[i, j] = sum((data[i] - data[j])**2)
    return D

- El keyword `cdef` define una variable con tipo estático (C)
- Los argumentos de las funciones también pueden definirse como tipo estático
- Se puede usar tipos de NumPy para entrada y salida con [memoryviews]()
    - Se declara un memory view con la sintáxis tipo [:, ::1] para una matriz de dos dimensones
- Se puede acelerar aun más deshabilitando las verificaciones de Python (por ejemplo boundcheck y wraparound)


In [ ]:
%%cython
import cython
import numpy as np

# Deshabilitamos las comprobaciones de Python:
@cython.boundscheck(False)
@cython.wraparound(False)
def distancia_pares_cython(double [:, ::1] data):
    # Usamos tipo estático con todas las variables:
    cdef int N = data.shape[0]
    cdef int M = data.shape[1]
    cdef int i, j, k
    cdef double tmp
    # Usamos un memory-view para nuestra variable de salida en formato NumPy
    cdef double [:, ::1] D = np.empty(shape=(N, N), dtype=np.double)
    # Hacemos un ciclo for "estilo C"
    for i in range(N):
        for j in range(N):
            tmp = 0.0
            for k in range(M):
                tmp += (data[i, k] - data[j, k])**2
            D[i, j] = tmp
    return D

El resultado es idéntico al del código NumPy pero considerablemente más veloz

In [ ]:
display(np.allclose(distancia_pares_numpy(data), distancia_pares_cython(data)))
%timeit -r3 -n1 distancia_pares_cython(data)

### Ejemplo: "Cythonizando" el fractal de Julia

Partiendo del código de Python puro del fractal de Julia, escribimos una versión en Cython

Recuerda:
- Usamos `cdef` para definir las variables con tipo estático
- También podemos usar `cdef` para definir una función que solo es llamada desde cython
- Usamos memory-views con el tipo adecuado para conectar con NumPy en la salida
- Deshabilitamos las comprobaciones de Python para ganar más velocidad

In [ ]:
%%cython
import cython
import numpy as np

cdef int evaluate_z(double zi, double zr, int maxiters, double cr=-0.835, double ci=-0.2321):
    cdef:
        int nit = 0
        double zi2 = zi**2
        double zr2 = zr**2
        
    while zi2 + zr2 <= 4. and nit < maxiters:
        zi = 2*zr*zi + ci
        zr = zr2 - zi2 + cr
        zr2 = zr**2
        zi2 = zi**2 
        nit +=1
    return nit

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def make_fractal_cython(int N, int maxiters=50):
    cdef:
        int [:, ::1] image = np.zeros(shape=(N, 2*N), dtype=np.int32) # el tipo es impotante!
        int i, j
    
    for i in range(N):
        for j in range(2*N):
            image[i, j] = evaluate_z(-1.+i*2./N, -2.+j*2./N, maxiters)
    return image

El resultado es idéntico al código secuencial en "Python puro"

In [ ]:
np.allclose(make_fractal(500), make_fractal_cython(500))

EL tiempo es considerablemente menor al código en "Python puro" y también al vectorizado con NumPy

In [ ]:
N = 500
%timeit -r3 -n1 make_fractal(N)
%timeit -r3 -n1 make_fractal_vectorized1(N)
%timeit -r3 -n1 make_fractal_cython(N)

Ahora podemos dibujar el fractal en mayor resolución en un tiempo razonable :)

In [ ]:
plt.figure(figsize=(10, 5), tight_layout=True)
plt.gca().axis('off')
plt.imshow(make_fractal(2000, 100));

# Usando Cython fuera de Jupyter



# Llamando una librería en C desde Cython

https://docs.python-guide.org/scenarios/clibs/

añadir links a documentación cython

https://clusterdata.nl/bericht/news-item/what-is-cython-python-at-the-speed-c/

https://www.infoworld.com/article/3250299/what-is-cython-python-at-the-speed-of-c.html

https://146.83.216.158/user/phuijse/files/Work/Books/Programming/OReilly%20Cython%2C%20A%20Guide%20for%20Python%20Programmers%20(2015).pdf

# Compilación Just-in-time (JIT) con Numba

Tema para otra iteración

In [ ]:
from numba import jit